In [106]:
import pandas as pd

df = pd.concat(pd.read_excel("Folds5x2_pp.xlsx", sheet_name=None), ignore_index=True)

train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

def normalize(df):
    for key in df.keys():
        df[key] = df[key] / df[key].max()
    return df.to_numpy()


train_features = normalize(train_dataset.drop(["PE"], axis=1))
test_features = normalize(test_dataset.drop(["PE"], axis=1))
train_labels = train_dataset["PE"].values.reshape(1, -1)[0]
test_labels = test_dataset["PE"].values.reshape(1, -1)[0]

In [107]:
import numpy as np

input_size = len(train_features[0])

# DAMN NETWORK
w1 = np.random.randint(1, 3, (input_size, 3))
b1 = np.random.randint(1, 3, (3, 1))
w2 = np.random.randint(1, 3, (3, 1))
b2 = np.random.randint(1, 3, (1, 1))

In [108]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def d_sigmoid(z):
    return sigmoid(z)*(1-sigmoid(z))

def relu(z):
    return np.maximum(z, 0)

def d_relu(z):
    z[z<=0] = 0
    z[z>0] = 1
    return z

def sqrt_mean_squared_error(x, y):
    return np.square(x - y).sum() / len(x)

In [135]:
# TRAINING DATA
# function to create a list containing mini-batches
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
  
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

# (4, 3)
# (3, 1)
# (3, 1)
# (1, 1)

def forward(x, w1, w2, b1, b2):
    Z1 = w1.T.dot(x) + b1
#     print(Z1.shape) #(3, 1)
#     print(w1.shape) #(4, 3)
#     print(x.shape) #(4, 1)
#     print(b1.shape) #(3, 1)
    
#     (3,1)
    A1 = relu(Z1)
#     Z2(1,1) = ((W2(3,m).T(m,3) * A1(3,m))(2,1) + b(1,1))
#     print(A1.shape) # (3, 1)
#     print(w2.shape) # (3, 1)
#     print(b2.shape) # (1, 1)
    Z2 = w2.T.dot(A1) + b2
#     A2(1,m) = A(Z2(1,m))
#     WE DON'T USE SIGMOID HERE TILL WE WANT A NUMBER AS RESULT
    print(Z2.shape)
    return Z2

# create batches that contains 50 (X,y) each
mini_batches = create_mini_batches(train_features, train_labels.reshape(len(train_labels), 1), 50) 

In [137]:
# THIS ENTIRE PROCESS COUNT AS AN EPOCH

# LEARNING_RATE
λ = 0.1

def gradient_descent(mini_batches, lw1, lw2, lb1, lb2):
    # mini_batch = zip(train_features, train_labels)
    for x, y in mini_batches[0:-2]:
        # GRADIENT TO BE
        dJdB1 = [np.zeros(b.shape) for b in lb1]
        dJdW1 = [np.zeros(w.shape) for w in lw1]
        dJdB2 = [np.zeros(b.shape) for b in lb2]
        dJdW2 = [np.zeros(w.shape) for w in lw2]

        # CALLED m IN SHAPES COMMENTS
        batch_size = len(x)

        # FORWARD PROPAGATION
        # Z1(3,m) = ((W1(4,3).T(3,4) * X(4,m))(3,m) + b(3,m))
        z1 = lw1.T.dot(x.T).T + lb1
        # A1(3,m) = A(Z1(3,m))
        a1 = sigmoid(z1)
        # Z2(1,1) = ((W2(3,m).T(m,3) * A1(3,m))(2,1) + b(1,1))
        z2 = (lw2.T.dot(a1.T) + lb2)
        # A2(1,m) = A(Z2(1,m))
        # WE DON'T USE SIGMOID HERE TILL WE WANT A NUMBER AS RESULT
        a2 = z2
        
        # BACKWARD PROPAGATION
        # HERE WE DO MATHS, NO LUCK, HIGHLY SUGGEST https://www.youtube.com/watch?v=tIeHLnjs5U8
        # dJdW2(3,m) = dZ2/dW2 * dA2/dZ2 * dJ/dA2 WITH J OUR COST FUNCTION (sqrt_mean_squared_error)
        dJdA2 = (2 * (a2 - y.T))
        dZ2dW2 = a1
        dJdW2 = dJdA2.dot(dZ2dW2).T
        dZ2dB2 = 1
        dJdB2 = ((dJdA2 * dZ2dB2)/batch_size).sum(axis=1)

        # HERE THE HARD PART IS ABOUT dJ/A1, which have impact on cost function thru Z2 -> A2 -> J
        # (1, m)
        # SINCE WE DON'T USE ACTIVATION FUNCTION ON LAST LAYER
        dJdZ2 = dJdA2
        # (3, m) = (3, m) * (3, m)
        dJdZ1 = (lw2.dot(dJdZ2)).T * d_sigmoid(z1)
        # (3, 4) = (3, m) * (m, 4)
        dJdW1 = dJdZ1.T.dot(x).T / batch_size
        # (1, 3) = (50, 3).sum(axis=y)
        dJdB1 = (dJdZ1 / batch_size).sum(axis=0)
        
        dJdW2 = dJdW2 / batch_size
        
        lw2 = lw2 + dJdW2
        lb2 = lb2 + dJdB2
        lw1 = lw1 + dJdW1
        lb1 = lb1 + dJdB1
        
        return lw1, lw2, lb1, lb2

# d = len(train_features)
# w1 = [w - λ/d * dw for w, dw in zip(w1, dJdW1)]
# w2 = [w - λ/d * dw for w, dw in zip(w2, dJdW2)]
# b1 = [b - λ/d * dw for b, dw in zip(b1, dJdB1)]
# b2 = [b - λ/d * dw for b, dw in zip(b2, dJdB2)]
# print(w1)

In [111]:
# nw1, nw2, nb1, nb2 = gradient_descent(mini_batches, w1, w2, b1, b2)
# print(nw1)
# print(nw2)
# print(nb1)
# print(nb2)

In [113]:
print(w1.shape)
print(w2.shape)

(4, 3)
(3, 1)


In [156]:
# THIS ENTIRE PROCESS COUNT AS AN EPOCH

# LEARNING_RATE
λ = 0.01

def gradient_descent(mini_batches, W1, W2, B1, B2):
    # GRADIENT TO BE
    dJdB1 = [np.zeros(b.shape) for b in B1]
    dJdW1 = [np.zeros(w.shape) for w in W1]
    dJdB2 = [np.zeros(b.shape) for b in B2]
    dJdW2 = [np.zeros(w.shape) for w in W2]
    for X, y in mini_batches:
        print(W1.shape)
        print(W2.shape)
        print(B1.shape)
        print(B2.shape)
        

        # CALLED m IN SHAPES COMMENTS
        # SHAPE AREN'T NECESSERALY IN GOOD ORDER SO (n,m) == (m,n) IN NOTATION
        batch_size = len(X)

        # FORWARD PROPAGATION
    
        # Z1(3,m) = ((W1(4,3).T(3,4) * X(4,m))(3,m) + b(3,m))
        Z1 = W1.T.dot(X.T) + B1
        # A1(3,m) = A(Z1(3,m))
        A1 = relu(Z1)
        
        # Z2(1,1) = ((W2(3,m).T(m,3) * A1(3,m))(2,1) + b(1,1))
        Z2 = W2.T.dot(A1) + B2
        # A2(1,m) = A(Z2(1,m))
        # WE DON'T USE ACTIVATION FUNCTION HERE TILL WE WANT A NUMBER AS RESULT
        
        # BACKWARD PROPAGATION
        
        # !! DOING MATHS, NO LUCK, HIGHLY SUGGEST https://www.youtube.com/watch?v=tIeHLnjs5U8 !!
        # dJdW2(3,m) = dZ2/dW2 * dJ/dZ2 WITH J OUR COST FUNCTION (sqrt_mean_squared_error)
        # WE USE Z2 BECAUSE NO ACTIVATION FUNCTION ON LAST LAYER
        # (1, m)
        dJdZ2 = Z2 * 2 *(Z2 - y.T)
        # A1 BECAUSE dJdW2 = dJdZ2 * dZ2dW2 WHERE dZ2dW2 = A1
        # (3,1)
        dJdW2 = dJdZ2.dot(A1.T) / batch_size
        # (1,1)
        dJdB2 = np.sum(dJdZ2, axis=1, keepdims=True) / batch_size

        # HERE THE HARD PART IS ABOUT dJ/A1, which have impact on cost function thru Z2 -> A2 -> J
        dJdZ1 = np.dot(W2, dJdZ2) * d_relu(Z1)
        dJdW1 = dJdZ1.dot(X) / batch_size
        dJdB1 = np.sum(dJdZ1, axis=1, keepdims=True) / batch_size
        
        W1 = W1 - λ * dJdW1.T     
        W2 = W2 - λ * dJdW2.T
        B1 = B1 - λ * dJdB1
        B2 = B2 - λ * dJdB2
        
        print(W1.shape)
        print(W2.shape)
        print(B1.shape)
        print(B2.shape)
        
        
    return W1, W2, B1, B2

W1, W2, B1, B2 = gradient_descent(mini_batches[:-3], w1, w2, b1, b2)

(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)

/tmp/ipykernel_154031/961552409.py:41: RuntimeWarning: overflow encountered in multiply
  dJdZ2 = Z2 * 2 *(Z2 - y.T)
/tmp/ipykernel_154031/961552409.py:49: RuntimeWarning: invalid value encountered in multiply
  dJdZ1 = np.dot(W2, dJdZ2) * d_relu(Z1)


(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)
(1, 1)
(4, 3)
(3, 1)
(3, 1)

In [157]:
test_feature = mini_batches[1][0][0].reshape(4,1)
test_label = mini_batches[1][1][0]
print(f"neural network predicted {forward(test_feature, W1, W2, B1, B2)}, should be {test_label}")

(1, 1)
neural network predicted [[nan]], should be [472.31]
